В качестве лабораторной работы по теме "Ансамбли моделей" предлагается участие в [соревановании на Kaggle](https://www.kaggle.com/competitions/ensembles-competition/leaderboard).


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib as plt
import matplotlib.pyplot as plt
%matplotlib inline

# **Основной вклад в общую работу: распарсить и очистить json, плюс провести лемматизацию текста, чтобы сохранить важные признаки**
## Параметры тоже подбирала, но в итоге использовали другие при сабмите

Из-за кодировки распаршивала json в Jypiter Notebook почему-то)
Аналогичный блок операций проделала для train
```
import pandas as pd
import ast

# Чтение CSV
train = pd.read_csv('train_contest.csv')
test = pd.read_csv('for_prediction.csv')

# Преобразуем строку в словарь и достаем нужные поля
test['city'] = test['area'].apply(lambda x: ast.literal_eval(x)['name'])
test['empl'] = test['employer'].apply(lambda x: ast.literal_eval(x)['name'])
test['type_vac'] = test['type'].apply(lambda x: ast.literal_eval(x)['id'])
test['exper'] = test['experience'].apply(lambda x: ast.literal_eval(x)['id'])

# Извлекаем все специализации в строку
test['special'] = test['specializations'].apply(
    lambda x: ', '.join([spec['name'] for spec in ast.literal_eval(x)]) if ast.literal_eval(x) else None
)
test['key'] = test['key_skills'].apply(
    lambda x: ', '.join([spec['name'] for spec in ast.literal_eval(x)]) if ast.literal_eval(x) else None
)
# Удаляем ненужные столбцы
test.drop(['area', 'type', 'experience', 'employer', 'specializations', 'snippet', 'address', 'alternate_url', 'url','key_skills', 'response_url','immediate_redirect_url','published_at', 'created_at'], axis=1, inplace=True)
test.drop(['sort_point_distance','archived','insider_interview', 'relations','working_days', 'working_time_intervals','working_time_modes'], axis=1, inplace=True)
test['timetable'] = test['schedule'].apply(lambda x: ast.literal_eval(x)['id'])
test[test.select_dtypes(include='bool').columns] = test.select_dtypes(include='bool').astype(int)
```



In [ ]:
train= pd.read_excel('train.xlsx')
test= pd.read_excel('predict.xlsx')

In [ ]:
train.head()

,premium,name,has_test,response_letter_required,accept_temporary,description,region,mean_salary,city,empl,type_vac,exper,special,key,timetable
0,0,Агент по недвижимости,0,0,0,"<p>Мы ищем людей, которым не достаточно того, ...",Санкт-Петербург,125000.0,Санкт-Петербург,АРИН,open,noExperience,"Агент, Жилье, Начальный уровень, Мало опыта, П...","Жилая недвижимость, Активные продажи, Подбор н...",flexible
1,0,IT-специалист/ техник,0,0,0,<ul> </ul> <p><em><strong>Навыки</strong></em>...,Иркутская область,50000.0,Братск,СМП-38,open,between1And3,"Начальный уровень, Мало опыта, Системный админ...",NaN,flyInFlyOut
2,0,Старший/ведущий инженер-программист,0,0,0,<strong>Обязанности:</strong> <ul> <li>Разрабо...,Ростовская область,80000.0,Ростов-на-Дону,"САРМАТ, НПП",open,between1And3,"Программирование, Разработка, Системы управлен...","Linux, Bash, C/C++",fullDay
3,0,Эксперт / методист приемной комиссии,0,0,0,<p>Департамент информационных технологий РХТУ ...,Москва,120000.0,Москва,РХТУ им. Д.И. Менделеева,open,between1And3,"Менеджмент продукта (Product manager), Управле...","1С программирование, Обновление конфигурации 1...",fullDay
4,0,Табельщица/табельщик,0,0,0,<p><strong>Обязанности:</strong></p> <ul> <li>...,Саратовская область,15000.0,Энгельс,"Сигнал, группа компаний",open,between1And3,Машиностроение,"Составление отчетности, Работа с оргтехникой, ...",fullDay


In [ ]:
import re
import pymorphy3
import pandas as pd
from nltk.corpus import stopwords

morph = pymorphy3.MorphAnalyzer()

russian_stop_words = stopwords.words('russian')

def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'[^а-яА-ЯёЁ\s]', '', text.lower())

        lemmatized_words = []
        for word in text.split():
            if word not in russian_stop_words:
                lemma = morph.parse(word)[0].normal_form
                lemmatized_words.append(lemma)
        return ' '.join(lemmatized_words)
    return text

for column in train.select_dtypes(include=['object']).columns:
    train[column] = train[column].apply(clean_text)

In [ ]:
for column in test.select_dtypes(include=['object']).columns:
    test[column] = test[column].apply(clean_text)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30072 entries, 0 to 30071
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   premium                   30072 non-null  int64  
 1   name                      30072 non-null  object 
 2   has_test                  30072 non-null  int64  
 3   response_letter_required  30072 non-null  int64  
 4   accept_temporary          30072 non-null  int64  
 5   description               30072 non-null  object 
 6   region                    30072 non-null  object 
 7   mean_salary               30072 non-null  float64
 8   city                      30072 non-null  object 
 9   empl                      30072 non-null  object 
 10  type_vac                  30072 non-null  object 
 11  exper                     30072 non-null  object 
 12  special                   30072 non-null  object 
 13  key                       24866 non-null  object 
 14  timeta

In [ ]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV

# Разделение признаков и целевой переменной
X = train.drop(columns=['mean_salary'])
y = train['mean_salary']
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

for feature in categorical_features:
    X[feature] = X[feature].fillna('missing')

# Настройка модели
model = CatBoostRegressor(verbose=0, cat_features=categorical_features)

# Словарь параметров для подбора
param_dist = {
    'iterations': [300],
    'learning_rate': [0.1, 0.05],
    'depth': [6],
    'l2_leaf_reg': [3, 5],
    'border_count': [64]
}

search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    scoring='neg_mean_squared_error',  # Для регрессии используем MSE
    n_iter=5,
    cv=2,
    verbose=2,
    n_jobs=1,
    random_state=42
)

# Подбор параметров
search.fit(X, y)

# Лучшая модель
best_model = search.best_estimator_
print("Параметры:", search.best_params_)

D:\Anaconda\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 4 is smaller than n_iter=5. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] END border_count=64, depth=6, iterations=300, l2_leaf_reg=3, learning_rate=0.1; total time=  17.8s
[CV] END border_count=64, depth=6, iterations=300, l2_leaf_reg=3, learning_rate=0.1; total time=  17.8s
[CV] END border_count=64, depth=6, iterations=300, l2_leaf_reg=3, learning_rate=0.05; total time=  17.4s
[CV] END border_count=64, depth=6, iterations=300, l2_leaf_reg=3, learning_rate=0.05; total time=  16.5s
[CV] END border_count=64, depth=6, iterations=300, l2_leaf_reg=5, learning_rate=0.1; total time=  16.7s
[CV] END border_count=64, depth=6, iterations=300, l2_leaf_reg=5, learning_rate=0.1; total time=  17.6s
[CV] END border_count=64, depth=6, iterations=300, l2_leaf_reg=5, learning_rate=0.05; total time=  16.5s
[CV] END border_count=64, depth=6, iterations=300, l2_leaf_reg=5, learning_rate=0.05; total time=  16.0s
Параметры: {'learning_rate': 0.1, 'l2_leaf_reg': 3, 'iterations': 300, 'depth': 6, 'border_count': 64}


CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=2]="помощник маркетолог": Cannot convert 'b'\xd0\xbf\xd0\xbe\xd0\xbc\xd0\xbe\xd1\x89\xd0\xbd\xd0\xb8\xd0\xba \xd0\xbc\xd0\xb0\xd1\x80\xd0\xba\xd0\xb5\xd1\x82\xd0\xbe\xd0\xbb\xd0\xbe\xd0\xb3'' to float

Параметры: **{'learning_rate': 0.1, 'l2_leaf_reg': 3, 'iterations': 300, 'depth': 6, 'border_count': 64}**

In [ ]:
X = train.drop(columns=['mean_salary'], errors='ignore')
y = train['mean_salary']

cat_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
for col in cat_features:
    X[col] = X[col].astype(str).fillna('missing').astype('category')


train_pool = Pool(data=X, label=y, cat_features=cat_features)
model = CatBoostRegressor(
    learning_rate=0.1,
    l2_leaf_reg=3,
    iterations=300,
    depth=6,
    border_count=64,
    cat_features=cat_features,
    verbose=50
)

model.fit(train_pool)

0:	learn: 101423.7733203	total: 55.7ms	remaining: 16.7s
50:	learn: 76953.3142046	total: 3.17s	remaining: 15.5s
100:	learn: 74490.8446523	total: 6.49s	remaining: 12.8s
150:	learn: 72834.9491767	total: 9.7s	remaining: 9.57s
200:	learn: 71512.1530893	total: 13s	remaining: 6.43s
250:	learn: 70362.6298456	total: 16.2s	remaining: 3.16s
299:	learn: 69375.8294791	total: 18.7s	remaining: 0us


In [ ]:
X_test = test.drop(columns=['Id'], errors='ignore')

for col in cat_features:
    if col in X_test.columns:
        X_test[col] = X_test[col].astype(str).fillna('missing').astype('category')

test_pool = Pool(data=X_test, cat_features=cat_features)

predictions = model.predict(test_pool)

submission = pd.DataFrame({
    'id': test['Id'],
    'prediction': predictions
})
submission.to_csv('submission.csv', index=False)